In [ ]:
import pymysql
import pandas as pd
import warnings

## Helpful functions

In [13]:
def decode_df(df):
    data_types = df.dtypes.to_dict()
    for col in df.columns:
        if data_types[col] == object:
            df[col] = df[col].str.decode('utf-8')
    return df

def connect(wiki_name, cnf_path = "/home/paws/.my.cnf"):
    return pymysql.connect(host = wiki_name + '.analytics.db.svc.wikimedia.cloud',
                           read_default_file = cnf_path,
                           database = wiki_name + '_p')

def pd_query(query, wiki):
    warnings.filterwarnings('ignore')
    connection = connect(wiki)
    return decode_df(pd.read_sql(query, connection))

def query_wikis(query, wikis):
    combined_result = pd.DataFrame()
    for wiki in wikis:
        result = pd_query(query, wiki)
        result['wiki_db'] = wiki
        combined_result = pd.concat([combined_result, result])
    return combined_result
    
# source: https://github.com/wikimedia/wmfdata-python/blob/main/wmfdata/utils.py
def sql_tuple(i):

    if type(i) != list:
        i = [x for x in i]

    if len(i) == 0:
        raise ValueError("Cannot produce an SQL tuple without any items.")

    list_repr = repr(i)
    return "(" + list_repr[1:-1] + ")"

### Misc code

In [ ]:
# with conn.cursor() as cur:
#     cur.execute(query)
#     data = cur.fetchall()
# conn.close()
# return data